In [1]:
import jetson.inference
import jetson.utils
import numpy as np

from jetbot import Camera
from jetbot import bgr8_to_jpeg
import ipywidgets.widgets as widgets
import traitlets
import cv2

In [2]:
from jetbot import Robot
robot = Robot()

In [3]:
class segmentationBuffers:
    def __init__(self, net):
        self.net = net
        self.mask = None
        self.overlay = None
        self.composite = None
        self.class_mask = None
        
        self.use_stats = True
        self.use_mask = True
        self.use_overlay = True
        self.use_composite = self.use_mask and self.use_overlay
        
        if not self.use_overlay and not self.use_mask:
            raise Exception("invalid visualize flags - valid values are 'overlay' 'mask' 'overlay,mask'")
             
        self.grid_width, self.grid_height = net.GetGridSize()	
        self.num_classes = net.GetNumClasses()

    @property
    def output(self):
        if self.use_overlay and self.use_mask:
            return self.composite
        elif self.use_overlay:
            return self.overlay
        elif self.use_mask:
            return self.mask
            
    def Alloc(self, shape, format):
        if self.overlay is not None and self.overlay.height == shape[0] and self.overlay.width == shape[1]:
            return

        if self.use_overlay:
            self.overlay = jetson.utils.cudaAllocMapped(width=shape[1], height=shape[0], format=format)

        if self.use_mask:
            mask_downsample = 2 if self.use_overlay else 1
            self.mask = jetson.utils.cudaAllocMapped(width=shape[1]/mask_downsample, height=shape[0]/mask_downsample, format=format) 

        if self.use_composite:
            self.composite = jetson.utils.cudaAllocMapped(width=self.overlay.width+self.mask.width, height=self.overlay.height, format=format) 

        if self.use_stats:
            self.class_mask = jetson.utils.cudaAllocMapped(width=self.grid_width, height=self.grid_height, format="gray8")
            self.class_mask_np = jetson.utils.cudaToNumpy(self.class_mask)
            
    def ComputeStats(self):
        if not self.use_stats:
            return
            
        # get the class mask (each pixel contains the classID for that grid cell)
        self.net.Mask(self.class_mask, self.grid_width, self.grid_height)

        # compute the number of times each class occurs in the mask
        class_histogram, _ = np.histogram(self.class_mask_np, self.num_classes)

        print('grid size:   {:d}x{:d}'.format(self.grid_width, self.grid_height))
        print('num classes: {:d}'.format(self.num_classes))

        print('-----------------------------------------')
        print(' ID  class name        count     %')
        print('-----------------------------------------')

        for n in range(self.num_classes):
            percentage = float(class_histogram[n]) / float(self.grid_width * self.grid_height)
            print(' {:>2d}  {:<18s} {:>3d}   {:f}'.format(n, self.net.GetClassDesc(n), class_histogram[n], percentage)) 
            
def mask_roi(img_th, roi):
    mask = np.zeros_like(img_th)
    cv2.fillPoly(mask, np.array([roi], np.int32), 255)
    masked_image = cv2.bitwise_and(img_th, mask)
    return masked_image

In [4]:
model = "fcn-resnet18-sun"
overlay_Alpha = 150
filter_mode = "point" # "linear", choices=["point", "linear"]
ignore_class = "toilet"

In [5]:
net = jetson.inference.segNet(model)
net.SetOverlayAlpha(overlay_Alpha)
buffers = segmentationBuffers(net)

In [6]:
capture = cv2.VideoCapture(1)
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 640)  # width
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 240) # height
capture.set(cv2.CAP_PROP_FPS, 10)

True

In [27]:
for i in range(3000):
    try:
        ret, img = capture.read()
        image_ori.value = bgr8_to_jpeg(img[:,:320,:])
        img = jetson.utils.cudaFromNumpy(img[:,:320,:])
        buffers.Alloc(img.shape, img.format)
        net.Process(img, ignore_class=ignore_class)
        if buffers.overlay:
            net.Overlay(buffers.overlay, filter_mode=filter_mode) 
        if buffers.mask:
            net.Mask(buffers.mask, filter_mode=filter_mode)
        if buffers.composite:
            jetson.utils.cudaOverlay(buffers.overlay, buffers.composite, 0, 0)
            jetson.utils.cudaOverlay(buffers.mask, buffers.composite, buffers.overlay.width, 0)

        img_rander = jetson.utils.cudaToNumpy(buffers.output)
        image_det.value = bgr8_to_jpeg(img_rander)
        mask_rander = jetson.utils.cudaToNumpy(buffers.mask)
    #     image_mask.value = bgr8_to_jpeg(mask_rander)
        floor_rander = mask_rander.copy()
        floor_rander[mask_rander[:,:,1]!=128] = 0
        floor_rander[mask_rander[:,:,0]!=0] = 0
        floor_rander[mask_rander[:,:,2]!=0] = 0
        n = np.sum(floor_rander)/128
        floor_mask = floor_rander.reshape((120,160,3))
        image_floor.value = bgr8_to_jpeg(floor_mask)

        # ROI 
        height, width, channel = floor_mask.shape
        roi = [(0, height),(50, height/2), (width-50, height/2),(width, height),]
        floor_mask_roi = mask_roi(floor_mask[:,:,1], roi)
        image_mask.value = bgr8_to_jpeg(floor_mask_roi)
        n_roi = np.sum(floor_mask_roi.reshape((height*width)))/128

        
        left_power = (0.15)
        right_power = (0.145)
        if n_roi>2200:
            pw = 1.0
            direction = "Straight"
            robot.set_motors(pw*left_power, pw*right_power)
        else:
            floor_mask_roi_left = floor_mask_roi[:,:int(0.5*width)].reshape((int(height*width/2)))
            n_roi_l = np.sum(floor_mask_roi_left)/128
            pw = 0.75
            if n_roi_l>n_roi-n_roi_l:
                direction = "left"
                robot.set_motors(-pw*left_power, pw*right_power)
            else:
                direction = "right"
                robot.set_motors(pw*left_power, -pw*right_power)
#             robot.stop()
        
        # print info
        print(f'count: {i}, floor: {n}, floor_roi: {n_roi}, stop: {n_roi<=2300}, direction: {direction}\t\t\t\t\t', end='\r')
    except:
        print('*********error jetbot stop*********',  end='\r')
        robot.stop()
robot.stop()

In [7]:
image_ori = widgets.Image(format='jpeg', width=320, height=240)
image_det = widgets.Image(format='jpeg', width=480, height=240)
image_floor = widgets.Image(format='jpeg', width=320, height=240)
image_mask = widgets.Image(format='jpeg', width=320, height=240)
image_box = widgets.HBox([image_ori, image_det, image_floor, image_mask])
display(image_box)

In [ ]:
capture.release() 